# ENTRENAMIENTO DEL MODELO
Vaig a fer rapid una base aon neteje rapid els nuls i prove diferents models para fer prediccions

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

In [116]:
data = pd.read_csv("temp.csv")
data = data[data["Marca"].str.lower() != "thuraya"]
data

,_id,Ano del modelo,Capacidad de almacenamiento digital,Marca,Tamano de la pantalla,Tipo de pantalla,precio_actual,precio_anterior,Potencia nominal de la bateria,Memoria RAM,Velocidad de CPU (GHz),numero_valoraciones,numero_estrellas,Profundidad (cm),Peso (g),ancho,alto
0,B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,245.00,199.99,5000.0,1.0,1.700000e+00,1034.0,3.7,1.27,77.11,540.0,960.0
1,B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,933.00,199.99,160.0,2.0,NaN,4.0,5.0,10.16,318.00,240.0,320.0
2,B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,279.99,279.99,4500.0,6.0,2.200000e+00,7752.0,4.4,0.77,179.00,1080.0,2400.0
3,B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,41.99,45.99,800.0,32.0,1.000000e-09,708.0,4.2,1.80,75.00,NaN,NaN
4,B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,48.67,48.67,800.0,16.0,NaN,296.0,4.0,1.90,74.00,3840.0,2160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,99.90,199.99,5000.0,4.0,NaN,8.0,4.7,2.57,254.42,1920.0,1080.0
949,B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,127.00,165.39,5000.0,6.0,2.050000e+00,16816.0,4.4,0.81,179.00,1920.0,1080.0
950,B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,299.00,199.99,5000.0,8.0,NaN,11.0,4.8,2.57,254.42,NaN,NaN
951,B0D8TCGKGN,2024.0,256.0,oukitel,6.52,hd,149.99,199.99,10600.0,24.0,1.600000e+00,659.0,4.4,2.57,254.42,1280.0,720.0


In [103]:
def top_marcas(df):
    """Devuelve las 5 marcas más frecuentes en el dataset."""
    return df["Marca"].value_counts().head(30)

top_marcas(data)

Marca
xiaomi       194
samsung      107
doogee        79
oukitel       60
oppo          41
motorola      39
blackview     32
realme        31
ulefone       29
apple         24
honor         19
cubot         19
vivo          16
google        15
oscal         13
umidigi       12
tcl           10
oneplus       10
msmga         10
zte            9
spc            8
fossibot       8
ttfone         6
panasonic      6
huawei         5
hammer h       5
zokoe          5
ciciglow       5
wrtogo         4
mcmgc          4
Name: count, dtype: int64

## NETEJA DE NULS (RAPIDO)

In [104]:
def fill_na_with_mode(df):
    
    df_filled = df.copy()
    for column in df_filled.columns:
        mode_value = df_filled[column].mode()[0] 
        df_filled[column].fillna(mode_value, inplace=True)
    return df_filled

In [105]:
data = fill_na_with_mode(data)


/tmp/ipykernel_7837/628468171.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled[column].fillna(mode_value, inplace=True)


## ELIMINAR IDS

In [106]:
def eliminar_columnas_no_deseadas(df):
    """Elimina columnas '_id' y cualquier columna no deseada como 'Unnamed'."""
    cols_a_eliminar = [col for col in df.columns if "Unnamed" in col or col == "_id"]
    df.drop(columns=cols_a_eliminar, inplace=True, errors='ignore')
    return df

In [107]:
data = eliminar_columnas_no_deseadas(data)
data


,Ano del modelo,Capacidad de almacenamiento digital,Marca,Tamano de la pantalla,Tipo de pantalla,precio_actual,precio_anterior,Potencia nominal de la bateria,Memoria RAM,Velocidad de CPU (GHz),numero_valoraciones,numero_estrellas,Profundidad (cm),Peso (g),ancho,alto
0,2013.0,256.0,samsung,4.30,lcd,245.00,199.99,5000.0,1.0,1.700000e+00,1034.0,3.7,1.27,77.11,540.0,960.0
1,2024.0,160.0,inmarsat,2.00,lcd,933.00,199.99,160.0,2.0,2.000000e+00,4.0,5.0,10.16,318.00,240.0,320.0
2,2024.0,256.0,samsung,6.70,superamoled,279.99,279.99,4500.0,6.0,2.200000e+00,7752.0,4.4,0.77,179.00,1080.0,2400.0
3,2015.0,128.0,ttfone,2.00,lcd,41.99,45.99,800.0,32.0,1.000000e-09,708.0,4.2,1.80,75.00,1080.0,1080.0
4,2015.0,128.0,doro,2.00,lcd,48.67,48.67,800.0,16.0,2.000000e+00,296.0,4.0,1.90,74.00,3840.0,2160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,2024.0,128.0,honor,6.56,lcd,99.90,199.99,5000.0,4.0,2.000000e+00,8.0,4.7,2.57,254.42,1920.0,1080.0
949,2024.0,128.0,xiaomi,6.43,amoled,127.00,165.39,5000.0,6.0,2.050000e+00,16816.0,4.4,0.81,179.00,1920.0,1080.0
950,2024.0,256.0,oppo,6.67,amoled,299.00,199.99,5000.0,8.0,2.000000e+00,11.0,4.8,2.57,254.42,1080.0,1080.0
951,2024.0,256.0,oukitel,6.52,hd,149.99,199.99,10600.0,24.0,1.600000e+00,659.0,4.4,2.57,254.42,1280.0,720.0


## CONVERTIR VARIABLES CATEGORICAS A NUMERICAS

In [108]:
def convertir_categoricas_a_numericas(df, categorical_cols):
    """Convierte variables categóricas en numéricas usando Label Encoding y guarda la referencia en un archivo de texto."""
    label_encoders = {}
    with open("referencia_marcas.txt", "w") as file:
        for col in categorical_cols:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
                label_encoders[col] = le
                for i, clase in enumerate(le.classes_):
                    file.write(f"{clase}:{i}\n")
    return df, label_encoders

In [109]:
# Definir columnas categóricas
categorical_cols = ["Marca", "Sistema operativo", "Tipo de pantalla"]
df, label_encoders = convertir_categoricas_a_numericas(data, categorical_cols)

In [110]:
# Separar features y target
X = df.drop(columns=["precio_actual"])
y = df["precio_actual"]

In [111]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## GRADIENT BOSSTING REGRESSOR

In [112]:

# Entrenar modelo GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)


# Entrenar modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


## EVALUACION

In [113]:

# Evaluar modelo GradientBoostingRegressor
gb_results = {
    "MAE": mean_absolute_error(y_test, y_pred_gb),
    "MSE": mean_squared_error(y_test, y_pred_gb),
    "R2 Score": r2_score(y_test, y_pred_gb),
}

# Evaluar modelo RandomForestRegressor
rf_results = {
    "MAE": mean_absolute_error(y_test, y_pred_rf),
    "MSE": mean_squared_error(y_test, y_pred_rf),
    "R2 Score": r2_score(y_test, y_pred_rf),
}

# Mostrar resultados
print("GradientBoostingRegressor:")
for metric, value in gb_results.items():
    print(f"  {metric}: {value}")

print("\nRandomForestRegressor:")
for metric, value in rf_results.items():
    print(f"  {metric}: {value}")

GradientBoostingRegressor:
  MAE: 119.93934025034923
  MSE: 200985.95765668582
  R2 Score: -2.8481645560680464

RandomForestRegressor:
  MAE: 103.1734222338569
  MSE: 89739.31099488177
  R2 Score: -0.718187876818488


## GUARDAR MODELO

In [114]:
def guardar_modelo(model, filename="modelo_entrenado.pkl"):
    """Guarda el modelo entrenado en un archivo local."""
    with open(filename, "wb") as file:
        pickle.dump(model, file)

In [115]:
guardar_modelo(rf_model)